# Linear Regression Demo

## Linear Regression using the California Housing Dataset

In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# We will use an example dataset that comes with scikit learn
from sklearn.datasets import fetch_california_housing

california = fetch_california_housing(as_frame=True)
print(california.data.shape)
print(california.target.shape)

(20640, 8)
(20640,)


In [14]:
np.array(california.data).shape

(20640, 8)

In [2]:
print(california.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [3]:
df = california.frame
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [ ]:
df.info()

In [ ]:
X = california.data
y = california.target

In [ ]:
X

In [ ]:
y

Let's split the data into train and test set!
We choose the test set to be 25% of the data.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1337)

In [ ]:
y_train.shape, y_val.shape, y_test.shape

For now, we will focus only on the features `['MedInc', 'Latitude', 'Longitude']` for simplicity. 

*Note*: In the real world, you would spend more time and brain power in this **feature selection** step!

In [ ]:
FEATURES = ['MedInc', 'Latitude', 'Longitude']
X_train = X_train[FEATURES]
X_val = X_val[FEATURES]
X_test = X_test[FEATURES]

We will use `pairplot` from the `seaborn` package (essentially a convenient `matplotlib` wrapper). This produces scatterplots between pairs of features. On the diagonal, we will plot a smoothed version of the *feature histogram*.

In [ ]:
import seaborn as sns

train_dataset = X_train.copy()[FEATURES]
train_dataset['MedHouseVal'] = y_train
sns.pairplot(train_dataset, kind="scatter", diag_kind="kde", plot_kws={'s': 3})

Let's create a plot that shows the median house value as a function of (latitude, longitude).

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=train_dataset,
    x='Longitude',
    y='Latitude',
    hue='MedHouseVal',
    size='MedHouseVal',
    palette='coolwarm',
    alpha=0.5,
)

plt.legend(title='MedHouseVal', loc='best')
plt.title('Median house value depending of\n their spatial location')
plt.show()

We will add the constant (dummy) feature to our design matrix $X$ (for the `train`, `validation` and the `test` set).

In [ ]:
X_train, X_val, X_test = X_train[FEATURES], X_val[FEATURES], X_test[FEATURES]
X_train['Constant'] = 1
X_val['Constant'] = 1
X_test['Constant'] = 1

X_train

Let's implement the *loss*
$$ \mathcal{L}(\theta) = \frac{1}{N} \| X \theta - y \|_2^2 $$

In [ ]:
def mean_squared_error(y_true, y_pred):
    squared_error = (y_pred - y_true)**2
    return np.mean(squared_error)

def loss(theta, X, y_true):
    y_pred = X @ theta
    return mean_squared_error(y_true, y_pred)

The analytical solution can be calculated using the `pinv` function (pseudoinverse) from `numpy`:

In [ ]:
from numpy.linalg import pinv 

def compute_theta_star_and_losses(X_train: np.ndarray, y_train: np.ndarray, 
                                  X_val: np.ndarray, y_val: np.ndarray):
    theta_star = pinv(X_train) @ y_train
    print(f'{theta_star=}')
    print("[Train] RMSE", np.sqrt(loss(theta_star, X_train, y_train)))
    print("[Validation] RMSE", np.sqrt(loss(theta_star, X_val, y_val)))

    return theta_star

## 1D Regression (using a single feature and a bias term)

To start, we will **only use the features** `MedInc` and the dummy feature `Constant` (to induce a bias term).

Let's calculate the analytical solution $\theta^*_0$ (we use the subscript because we will compute other solutions soon). 


In [ ]:
theta_star_0 = compute_theta_star_and_losses(X_train[['MedInc', 'Constant']].to_numpy(), y_train.to_numpy(), 
                                             X_val[['MedInc', 'Constant']].to_numpy(), y_val.to_numpy())


Thus, our regression function $f_\theta$ reads
$$ f_{\theta_0^*}(\text{MedInc}) = 0.42 \cdot \text{MedInc} + 0.46$$

Let's plot the regression line using the validation set (since it's 1D):

In [ ]:
medinc = X_val['MedInc'].to_numpy()
x_0, x_max = np.array([0, 1]), np.array([np.max(medinc), 1])
y_0 = x_0.T @ theta_star_0
y_max = x_max.T @ theta_star_0

plt.figure(figsize=(8, 6))
plt.scatter(medinc, y_val.to_numpy(), s=5)
plt.plot([0, np.max(medinc)], [y_0, y_max], c='r', linestyle='--', label='Regression line')
plt.xlabel('MedInc')
plt.ylabel('MedHouseVal')

plt.title('MedInc vs MedHouseVal (Validation Set)')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import r2_score

def plot_true_vs_pred(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mse = np.mean((y_true - y_pred)**2)
    rmse = np.sqrt(mse)
    m = np.max([np.max(y_true), np.max(y_pred)])

    plt.figure(figsize=(8, 8))
    plt.plot([0, m], [0, m], c='r')
    plt.scatter(y_pred, y_true, s=5)
    plt.xlabel('y_pred')
    plt.ylabel('y_true')
    plt.title(f'Predictions vs True Targets\nR^2 = {np.round(r2, 3)}, RMSE = {np.round(rmse, 3)}')
    plt.show()

We can scatter plot `y_pred` vs `y_true`. If we used the *perfect* regressor (`y_pred == y_true` for all samples) this plot would show the *identity function* here ($45^\circ$ line).

In [ ]:
y_pred_val = X_val[['MedInc', 'Constant']] @ theta_star_0
plot_true_vs_pred(y_true=y_val, y_pred=y_pred_val)

Also, we show the `R^2` score (coefficient of determination) for the validation set, which is defined as

$$ R^2 = 1 - \frac{\sum_{i=1}^N (y^{(i)} - \hat{y}^{(i)})^2}{\sum_{i=1}^N (y^{(i)} - \bar{y})^2} $$

where $f_\theta(\mathbf{x}^{(i)}) = \hat{y}^{(i)}$ are the predictions and $\bar{y} = \frac{1}{N} \sum_{i=1}^N y_i$ is the mean of the target values.

## Adding more features!

Let's add the features `Latitude` and `Longitude`!

In [ ]:
theta_star_1 = compute_theta_star_and_losses(X_train.to_numpy(), y_train.to_numpy(), 
                                             X_val.to_numpy(), y_val.to_numpy())

Our regression function $f_\theta$ now reads
$$ f_{\theta_1^*}(\text{MedInc}, \text{Lat}, \text{Lon}) = 0.36 \cdot \text{MedInc} - 0.49 \cdot \text{Lat} - 0.50 \cdot \text{Lon} - 41.90$$

In [ ]:
y_pred_val = X_val @ theta_star_1
plot_true_vs_pred(y_true=y_val, y_pred=y_pred_val)

## Using non-linear feature transformations

We can transform our *features* (read $\text{MedInc}, \text{Lat}, \text{Lon}$) using some non-linear functions $\phi_{j=1,\dots,M}$.

For example, we will define $\phi(\text{Lat}, \text{Lon}) = \min(\text{Distance to Los Angeles}, \text{Distance to San Francisco})$

In [ ]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians as rad

EARTH_RADIUS = 6_378_000 # earth radius in meters
san_francisco_coords = (rad(37.7749), rad(-122.4194)) # thanks OpenStreetMap!
los_angeles_coords = (rad(34.0549), rad(-118.2426)) # thanks OpenStreetMap!

def distance_to_city(coords, city):
    assert city in ['los_angeles', 'san_francisco']
    city_coords = los_angeles_coords if city == 'los_angeles' else san_francisco_coords

    dists = []
    for i in range(coords.shape[0]):
        coords_row = (rad(coords[i, 0]), rad(coords[i, 1]))
        dist = haversine_distances([coords_row, city_coords])
        dist = (dist[0, 1] * EARTH_RADIUS) / 1000 # distance in km
        dists.append(dist)
    
    return np.array(dists)

# non-linear feature transform
def add_min_city_distances_feature(X: pd.DataFrame):
    coords = X[['Latitude', 'Longitude']].to_numpy()
    dist_to_la = distance_to_city(coords, city='los_angeles')
    dist_to_sf = distance_to_city(coords, city='san_francisco')
    min_dists = np.vstack([dist_to_la, dist_to_sf]).min(axis=0)
    X = X.copy() # create a copy such that we don't modify the input DataFrame
    X['MinCityDistances'] =  min_dists

    return X

In [ ]:
X_train_augmented = add_min_city_distances_feature(X_train)
X_val_augmented = add_min_city_distances_feature(X_val)
X_test_augmented = add_min_city_distances_feature(X_test)

X_train_augmented

In [ ]:
plt.scatter(X_train_augmented['MinCityDistances'], y_train, s=5)
plt.xlabel('MinCityDistances')
plt.ylabel('MedHouseVal')
plt.show()

In [ ]:
theta_star_2 = compute_theta_star_and_losses(X_train_augmented, y_train, X_val_augmented, y_val)

Thus, we have

$$ f_{\theta_2^*}(\text{MedInc}, \text{Lat}, \text{Lon}) = 0.36 \cdot \text{MedInc} - 0.36 \cdot \text{Lat} - 0.38 \cdot \text{Lon} - 0.002 \cdot \phi(\text{Lat}, \text{Lon}) - 32.60$$

This is *still linear* in $\theta$ (it's just non-linear in the features)!

In [ ]:
y_pred_val_2 = X_val_augmented @ theta_star_2
plot_true_vs_pred(y_true=y_val, y_pred=y_pred_val_2)

# Some more non-linear feature transformations

Let's add some more non-linearly transformed features using *basis functions*!
For example, 
$$\phi_1(x) = x^2 \qquad \text{and} \qquad \phi_5(x) = \exp\left(\frac{-(x - 1)^2}{2\sigma^2}\right)$$

In [ ]:
sigma = 1.

basis_functions = [
    # Polynomial basis functions
    lambda x: x ** 2,
    lambda x: x ** 3,
    lambda x: x ** 4,
    lambda x: x ** 5,

    # Radial Basis Functions
    lambda x: np.exp(-((x - 1) ** 2) / (2 * sigma ** 2)),
    lambda x: np.exp(-((x - 3) ** 2) / (2 * sigma ** 2)),
    lambda x: np.exp(-((x - 5) ** 2) / (2 * sigma ** 2)),
    lambda x: np.exp(-((x - 7) ** 2) / (2 * sigma ** 2)),
    lambda x: np.exp(-((x - 9) ** 2) / (2 * sigma ** 2)),
]

In [ ]:
def add_basis_function_features(X: np.ndarray):
    X_nonlin_features = []
    for basis_fn in basis_functions:
        X_nonlin_features.append(basis_fn(X))
    
    X_nonlin_features = np.hstack(X_nonlin_features)
    return np.hstack([X, X_nonlin_features])

In [ ]:
X_train_augmented_basis = add_basis_function_features(X_train_augmented.to_numpy())
X_val_augmented_basis = add_basis_function_features(X_val_augmented.to_numpy())
X_test_augmented_basis = add_basis_function_features(X_test_augmented.to_numpy())

In [ ]:
X_train_augmented_basis.shape

In [ ]:
theta_star_3 = compute_theta_star_and_losses(X_train_augmented_basis, y_train, X_val_augmented_basis, y_val)

In [ ]:
y_pred_val_3 = X_val_augmented_basis @ theta_star_3
plot_true_vs_pred(y_true=y_val, y_pred=y_pred_val_3)

# Model Selection

So, which of the four models do we finally choose?

+ $f_{\theta_0^*}(\text{MedInc})$ with $R^2 = 0.48$ and $RMSE = 0.83$
    + Too simple, underfitting
+ $f_{\theta_1^*}(\text{MedInc}, \text{Lat}, \text{Lon})$ with $R^2 = 0.59$ and $RMSE = 0.75$
    + Decent fit, still simple and interpretable
+ $f_{\theta_2^*}(\text{MedInc}, \text{Lat}, \text{Lon})$ with $R^2 = 0.60$ and $RMSE = 0.73$
    + Decent fit, includes manual feature engineering, but is still interpretable
+ $f_{\theta_3^*}(\text{MedInc}, \text{Lat}, \text{Lon})$ with $R^2 = 0.64$ and $RMSE = 0.69$
    + Best fit, but is less interpretable (more complex)

Let's pick the third model $f_{\theta_2^*}(\text{MedInc}, \text{Lat}, \text{Lon})$ as our final model and calculate the test set performance.

In [ ]:
y_pred_test = X_test_augmented @ theta_star_2
plot_true_vs_pred(y_true=y_test, y_pred=y_pred_test)

**Important**: We do not touch the test set until the very end of our model selection process!

We don't want to use the test set to make *any* decisions about our model. It should only be used to assess the final model performance.